In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv("../etl/train_dataset_after_pca.csv")
eval_df = pd.read_csv("../etl/val_dataset_after_pca.csv")

In [ ]:
from envs.gym_market_env import CustomEnv

In [ ]:
from stable_baselines3.common.env_checker import check_env

In [ ]:
discount = 0.2
reward_multiplicator = 144.5
negative_reward_multiplicator = 140.6

train_py_env = CustomEnv(
    trades = train,
    features = [c for c in train.columns.values if "f_" in c] + ["feature_0", "weight"],
    reward_column = "resp",
    weight_column = "weight",
    include_weight=True,
    reward_multiplicator = reward_multiplicator,
    negative_reward_multiplicator = negative_reward_multiplicator
)

In [ ]:
check_env(train_py_env)

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO



In [ ]:
env = DummyVecEnv([lambda: train_py_env])


In [ ]:
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=20000)
obs = env.reset()
for i in range(2000):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()